<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://github.com/gee-community/geemap/tree/master/tutorials/ImageCollection/02_image_collection_metadata.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/gee-community/geemap/blob/master/tutorials/ImageCollection/02_image_collection_metadata.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/gee-community/geemap/blob/master/tutorials/ImageCollection/02_image_collection_metadata.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# ImageCollection Information and Metadata
As with Images, there are a variety of ways to get information about an `ImageCollection`. The collection can be printed directly to the console, but the console printout is limited to 5000 elements. Collections larger than 5000 images will need to be filtered before printing. Printing a large collection will be correspondingly slower. The following example shows various ways of getting information about image collections programmatically:

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/gee-community/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/gee-community/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.foliumap`](https://github.com/gee-community/geemap/blob/master/geemap/foliumap.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print("geemap package not installed. Installing ...")
    subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.foliumap as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
The default basemap is `Google Satellite`. [Additional basemaps](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py#L13) can be added using the `Map.add_basemap()` function. 

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap("ROADMAP")  # Add Google Map
Map

## Add Earth Engine Python script 

In [ ]:
# Load a Landsat 8 ImageCollection for a single path-row.
collection = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
    .filter(ee.Filter.eq("WRS_PATH", 44))
    .filter(ee.Filter.eq("WRS_ROW", 34))
    .filterDate("2014-03-01", "2014-08-01")
)
print("Collection: ", collection.getInfo())

In [ ]:
# Get the number of images.
count = collection.size()
print("Count: ", count.getInfo())

In [ ]:
# Get the date range of images in the collection.
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
print(
    "Date range: ",
    ee.Date(range.get("min")).getInfo(),
    ee.Date(range.get("max")).getInfo(),
)

In [ ]:
# Change the date format.
range = collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
start_date = ee.Date(range.get("min")).format("YYYY-MM-dd")
end_date = ee.Date(range.get("max")).format("YYYY-MM-dd")
print("Date range: ", start_date.getInfo(), end_date.getInfo())

In [ ]:
# Get statistics for a property of the images in the collection.
sunStats = collection.aggregate_stats("SUN_ELEVATION")
print("Sun elevation statistics: ", sunStats.getInfo())

In [ ]:
# Get statistics for a property of the images in the collection.
clouds = collection.aggregate_array("CLOUD_COVER")
print("Cloud covers: ", clouds.getInfo())

In [ ]:
# Sort by a cloud cover property, get the least cloudy image.
image = ee.Image(collection.sort("CLOUD_COVER").first())
print("Least cloudy image: ", image.propertyNames().getInfo())
print("\nCloud cover: ", image.get("CLOUD_COVER").getInfo())

In [ ]:
# Limit the collection to the 10 most recent images.
recent = collection.sort("system:time_start", False).limit(10)
images = recent.aggregate_array("system:index").getInfo()
print("Recent images:")
for image in images:
    print(image)

## Display Earth Engine data layers 

In [ ]:
Map.addLayerControl()
Map